In [4]:
from evolution import genetic_algorithm as ga
import numpy as np
import string

In [15]:
char_set = string.ascii_letters + string.digits

length = 37

random_string = ''.join(np.random.choice(list(char_set), size=length))

print(random_string)

Nor5oC0QpgYE8IijkH1vw08ZBB1ORFRJ6X5I1


In [64]:
goal = "Não, não sei não, to sem criatividade"

def generator():
    length = len(goal)
    return ''.join(np.random.choice(list(char_set), size=length))

def dist(candidate):
    d = 0
    for i,c in enumerate(candidate):
        d+=abs(ord(c)-ord(goal[i]))
    
    return d

s1 = generator()
s2 = generator()

print(s1,dist(s1))
print(s2,dist(s2))


#ind = ga.Ind()

6OW0Fu5xIlrr1AL9seHb5OrizmmjU815r2Rz0 1484
OjazC8yOiOLW16rHzBv0ms50U9dfjxXYU929u 1521
